In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [2]:
total_train = pd.read_csv('postulaciones_nopostulacionesRedCodificado.csv')

In [3]:
total_train.count()

Unnamed: 0              8035569
idaviso                 8035569
idpostulante            8035569
nombre_sort             8035569
se_postulo              8035569
estadoEncoded           8035569
tipoDeTrabajoEncoded    8035569
nivelLaboralEncoded     8035569
sexoEncoded             8035569
edadEncoded             8035569
zonaEncoded             7474825
dtype: int64

In [3]:
total_train = total_train.drop(columns = ['idaviso','idpostulante','Unnamed: 0'])

In [4]:
total_train['zonaEncoded'] = total_train['zonaEncoded'].fillna(method = 'pad')

In [5]:
total_train.head()

,nombre_sort,se_postulo,estadoEncoded,tipoDeTrabajoEncoded,nivelLaboralEncoded,sexoEncoded,edadEncoded,zonaEncoded
0,6.0,1,0,0,0,2,3.0,0.0
1,6.0,1,1,0,0,1,3.0,0.0
2,6.0,1,0,0,0,0,0.0,0.0
3,6.0,1,1,0,0,0,3.0,0.0
4,6.0,1,1,0,0,0,2.0,0.0


In [6]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column in total_train.columns:
    if total_train[column].dtype == type(object):
        total_train[column] = le.fit_transform(total_train[column].astype(str))

total_train.sample(5)

,nombre_sort,se_postulo,estadoEncoded,tipoDeTrabajoEncoded,nivelLaboralEncoded,sexoEncoded,edadEncoded,zonaEncoded
6254293,2.0,0,1,0,0,0,3.0,0.0
2656780,3.0,1,0,1,0,0,0.0,0.0
5327026,1.0,1,1,0,0,0,0.0,0.0
566694,1.0,1,1,0,0,0,1.0,0.0
3150155,3.0,1,0,0,0,0,2.0,0.0


In [7]:
#Normalizamos los valores
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(total_train) 
total_train.loc[:,:] = scaled_values

In [8]:
total_train.head()

,nombre_sort,se_postulo,estadoEncoded,tipoDeTrabajoEncoded,nivelLaboralEncoded,sexoEncoded,edadEncoded,zonaEncoded
0,1.0,1.0,0.0,0.0,0.0,1.0,0.75,0.0
1,1.0,1.0,0.5,0.0,0.0,0.5,0.75,0.0
2,1.0,1.0,0.0,0.0,0.0,0.0,0.00,0.0
3,1.0,1.0,0.5,0.0,0.0,0.0,0.75,0.0
4,1.0,1.0,0.5,0.0,0.0,0.0,0.50,0.0


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(total_train.drop(['se_postulo'], axis=1).values, total_train['se_postulo'], test_size=0.33, random_state=42)

In [23]:
#Crear Arbol de decision con profundidad = DEPENDE DE QUE DE LA CELDA ANTERIOR
decision_tree = tree.DecisionTreeClassifier(criterion='gini',max_depth = 6)

In [24]:

decision_tree.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [25]:
decision_tree.predict(X_test)

array([1., 1., 1., ..., 1., 1., 1.])

In [26]:
#Pruebo el score para ver si realiza overfitting
decision_tree.score(X_test,y_test)


0.7560230309329202

In [27]:
test = pd.read_csv('test_encoded.csv')

In [28]:
test['zonaEncoded'] = test['zonaEncoded'].fillna(method = 'bfill')

In [79]:
test.count()

id                      100000
idaviso                 100000
idpostulante            100000
nombre_sort             100000
estadoEncoded           100000
tipoDeTrabajoEncoded    100000
nivelLaboralEncoded     100000
sexoEncoded             100000
edadEncoded             100000
zonaEncoded             100000
dtype: int64

In [29]:
test = test.drop(['idaviso','idpostulante'],axis=1)

In [30]:
result = decision_tree.predict(test.drop(['id'],axis = 1))

In [31]:
test['sepostulo'] = result

In [32]:
drop_elem = ['estadoEncoded','nombre_sort','tipoDeTrabajoEncoded','nivelLaboralEncoded','sexoEncoded','edadEncoded','zonaEncoded']
test = test.drop(drop_elem,axis=1)

In [33]:
#Verificamos como nos fue aproximadamente
test['sepostulo'].value_counts()

1.0    99451
0.0      549
Name: sepostulo, dtype: int64

In [34]:
FirstDecisionTree = test.to_csv('DecisionTreeResult4Normalizados.csv', index=False)